In [1]:
import os
import cv2
import shutil
from sklearn.model_selection import train_test_split

In [7]:
data_dir = r'data\FF++'
output_dir = r'data\FF++_split_frames'
img_size = (224,224)
batch_size = 32

# Extracting Frames

In [3]:
os.path.splitext(os.path.basename("data/FF++\fake\01_02__outside_talking_still_laughing__YVGY8LOK.mp4"))

('FF++\x0cake\x01_02__outside_talking_still_laughing__YVGY8LOK', '.mp4')

In [4]:
def extract_frames(video, save_dir, num_frames=5):
    cap = cv2.VideoCapture(video)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_ids = [(total_frames*i)//num_frames for i in range(num_frames)]

    flag=True
    for count, id_ in enumerate(frame_ids):
        cap.set(cv2.CAP_PROP_POS_FRAMES, id_)
        ret, frame = cap.read()
        if ret:
            frame_filename = f"{os.path.splitext(os.path.basename(video))[0]}_{count}.jpg"
            save_path = os.path.join(save_dir, frame_filename)
            cv2.imwrite(save_path, frame)
        else:
            flag = False
    cap.release()
    print(f"Extracted frames from {video}" if flag else "Failed to extract frames.")

# Splitting Dataset

In [5]:
def split_data(class_name, split_ratio=(0.7, 0.15, 0.15)):
    source = os.path.join(data_dir, class_name)
    all_files = os.listdir(source)

    train_files, val_test_files = train_test_split(all_files, train_size=split_ratio[0], random_state=42)
    val_files, test_files = train_test_split(val_test_files, test_size=0.5, random_state=42)
    
    for split, files in zip(['train', 'val', 'test'],
                            [train_files, val_files, test_files]):
        print(f"{split.upper()} set: {len(files)} videos")
        save_dir = os.path.join(output_dir, split, class_name)
        os.makedirs(save_dir, exist_ok=True)
        for file in files:
            extract_frames(os.path.join(source, file), save_dir)

In [6]:
for cls in ['real', 'fake']:
    split_data(cls)

TRAIN set: 140 videos
Extracted frames from data\FF++\real\13__walk_down_hall_angry.mp4
Extracted frames from data\FF++\real\07__walk_down_hall_angry.mp4
Extracted frames from data\FF++\real\02__walk_down_hall_angry.mp4
Extracted frames from data\FF++\real\01__walking_down_indoor_hall_disgust.mp4
Extracted frames from data\FF++\real\03__kitchen_still.mp4
Extracted frames from data\FF++\real\10__kitchen_still.mp4
Extracted frames from data\FF++\real\03__talking_angry_couch.mp4
Extracted frames from data\FF++\real\15__outside_talking_pan_laughing.mp4
Extracted frames from data\FF++\real\07__outside_talking_still_laughing.mp4
Extracted frames from data\FF++\real\11__talking_angry_couch.mp4
Extracted frames from data\FF++\real\04__outside_talking_pan_laughing.mp4
Extracted frames from data\FF++\real\10__walk_down_hall_angry.mp4
Extracted frames from data\FF++\real\13__secret_conversation.mp4
Extracted frames from data\FF++\real\03__talking_against_wall.mp4
Extracted frames from data\FF++\r